### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [84]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.Phases.PHASE_ORDER


In [87]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [89]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

model_phases = dr.Phases.PHASE_ORDER

label_type = "stai"
threshold = "fixed"

x, y = train.Train_WESAD.get_wesad_data(
    metrics, model_phases, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.86068306 0.88502281 0.89363103 0.89557051 0.89669385 0.86905019
 0.88478179 0.88670178 0.8899848  0.89841301 0.8674841  0.8804441
 0.88968767 0.89580356 0.89951736 0.86611379 0.87658378 0.88117315
 0.88447588 0.8896475  0.85798638 0.87827494 0.88119928 0.87800653
 0.88820858        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [57 13]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [50 20]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [51 19]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [50 20]

None


[('bpm', 164), ('rmssd', 108), ('sdnn', 108), ('lf_rr', 82), ('hf_rr', 66)]


[('bpm', 0.4668565709320435), ('rmssd', 0.17997908673587615), ('sdnn', 0.13589589071652938), ('hf_rr', 0.1323147213487

One or more of the test scores are non-finite: [0.87873631 0.91726842 0.91948554 0.91772739 0.92773251 0.89479626
 0.92299732 0.92193457 0.92842067 0.92878528 0.89542914 0.92471275
 0.92195849 0.92539505 0.92680133 0.88397248 0.90673    0.90999161
 0.9140221  0.91872673 0.89351652 0.90452494 0.91295941 0.91586828
 0.92119407        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [62  8], Predictions: [0 1], [64  6]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [62  8], Predictions: [0 1], [55 15]
Training RF ...
Model RF, Actual: [0 1], [62  8], Predictions: [0 1], [57 13]
Training XGB ...
Model XGB, Actual: [0 1], [62  8], Predictions: [0 1], [52 18]

None


[('bpm', 149), ('rmssd', 126), ('sdnn', 95), ('hf_rr', 93), ('lf_rr', 74)]


[('bpm', 0.4730874151005053), ('rmssd', 0.1994606349877523), ('hf_rr', 0.12397162370441454), ('sdnn', 0.12275319627316647), 

...

Grid search for XGB ...
SVM: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [62  8], Predictions: [0 1], [64  6]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [62  8], Predictions: [0 1], [63  7]
Training RF ...
Model RF, Actual: [0 1], [62  8], Predictions: [0 1], [64  6]
Training XGB ...
Model XGB, Actual: [0 1], [62  8], Predictions: [0 1], [62  8]

None


[('bpm', 168), ('rmssd', 122), ('hf_rr', 91), ('sdnn', 89), ('lf_rr', 64)]


[('bpm', 0.4488860312494252), ('rmssd', 0.20164777781788018), ('hf_rr', 0.13652207160197885), ('sdnn', 0.12352090133455113)

One or more of the test scores are non-finite: [0.85099764 0.85451449 0.86467725 0.86464138 0.87577664 0.85468317
 0.85786643 0.86885803 0.86885909 0.87985293 0.84704184 0.86625546
 0.87608189 0.87767794 0.88883902 0.85868977 0.87538262 0.88569182
 0.88995048 0.89754606 0.85013421 0.86922374 0.88557016 0.88880321
 0.89029067        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [61  8], Predictions: [0 1], [62  7]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [61  8], Predictions: [0 1], [62  7]
Training RF ...
Model RF, Actual: [0 1], [61  8], Predictions: [0 1], [62  7]
Training XGB ...
Model XGB, Actual: [0 1], [61  8], Predictions: [0 1], [63  6]

None


[('bpm', 149), ('rmssd', 128), ('sdnn', 90), ('hf_rr', 83), ('lf_rr', 64)]


[('bpm', 0.4342487098254582), ('rmssd', 0.20218824170429855), ('hf_rr', 0.14052246174278243), ('sdnn', 0.13795788443842644),

One or more of the test scores are non-finite: [0.847957   0.86366306 0.87548824 0.87320083 0.87245235 0.85818384
 0.86887579 0.88180588 0.88386967 0.88055501 0.8540271  0.86000983
 0.88658267 0.88412575 0.88316706 0.84236085 0.86850167 0.89071914
 0.88683824 0.88629253 0.84705493 0.86960864 0.89433607 0.88231811
 0.88394202        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [61  8], Predictions: [0 1], [62  7]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [61  8], Predictions: [0 1], [61  8]
Training RF ...
Model RF, Actual: [0 1], [61  8], Predictions: [0 1], [62  7]
Training XGB ...
Model XGB, Actual: [0 1], [61  8], Predictions: [0 1], [63  6]

None


[('bpm', 158), ('rmssd', 124), ('sdnn', 81), ('hf_rr', 79), ('lf_rr', 69)]


[('bpm', 0.4310644377769634), ('rmssd', 0.22394553613884313), ('sdnn', 0.14046366783937791), ('hf_rr', 0.12594348873877256), 